# Pipeline

In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer

In [2]:
!ls data

submission_format.csv
test_data.csv
test_set_features.csv
training_data.csv
training_set_features.csv
training_set_labels.csv


Load in training data for initial ETL:

In [3]:
#import training data
train = pd.read_csv("./data/training_data.csv")
train

,Unnamed: 0,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,25194,25194,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,1.0,1.0,NaN,NaN,0,0
1,14006,14006,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,NaN,Employed,lzgpxyit,"MSA, Not Principle City",2.0,1.0,fcxhlnwr,oijqvulv,0,1
2,11285,11285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,kbazzjca,"MSA, Principle City",0.0,1.0,wlfvacwt,hfxkjkmi,0,1
3,2900,2900,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,mlyzmhmf,"MSA, Not Principle City",0.0,0.0,mcubkhph,ukymxvdu,0,0
4,19083,19083,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,NaN,NaN,bhuqouqj,"MSA, Not Principle City",NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,21575,21575,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,Own,Not in Labor Force,qufhixun,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
20026,5390,5390,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,Own,Unemployed,mlyzmhmf,"MSA, Principle City",0.0,0.0,NaN,NaN,0,0
20027,860,860,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,Non-MSA,1.0,0.0,atmlpfrs,xqwwgdyp,0,0
20028,15795,15795,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,Own,Employed,kbazzjca,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,0,0


In [4]:
#remove unnamed:0 becuase its a duplicate of respondent_id
train = train.drop(labels="Unnamed: 0", axis=1)

In [5]:
train

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,25194,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,1.0,1.0,NaN,NaN,0,0
1,14006,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,NaN,Employed,lzgpxyit,"MSA, Not Principle City",2.0,1.0,fcxhlnwr,oijqvulv,0,1
2,11285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,kbazzjca,"MSA, Principle City",0.0,1.0,wlfvacwt,hfxkjkmi,0,1
3,2900,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,mlyzmhmf,"MSA, Not Principle City",0.0,0.0,mcubkhph,ukymxvdu,0,0
4,19083,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,NaN,NaN,bhuqouqj,"MSA, Not Principle City",NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,21575,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,Own,Not in Labor Force,qufhixun,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
20026,5390,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,Own,Unemployed,mlyzmhmf,"MSA, Principle City",0.0,0.0,NaN,NaN,0,0
20027,860,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,Own,Employed,qufhixun,Non-MSA,1.0,0.0,atmlpfrs,xqwwgdyp,0,0
20028,15795,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,Own,Employed,kbazzjca,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,0,0


In [6]:
x_train = train.drop(labels=["respondent_id","h1n1_vaccine","seasonal_vaccine"], axis=1)
y_train = train["seasonal_vaccine"]

In [7]:
x_train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,1.0,1.0,NaN,NaN
1,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,Married,NaN,Employed,lzgpxyit,"MSA, Not Principle City",2.0,1.0,fcxhlnwr,oijqvulv
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,kbazzjca,"MSA, Principle City",0.0,1.0,wlfvacwt,hfxkjkmi
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Own,Employed,mlyzmhmf,"MSA, Not Principle City",0.0,0.0,mcubkhph,ukymxvdu
4,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,bhuqouqj,"MSA, Not Principle City",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,"> $75,000",Not Married,Own,Not in Labor Force,qufhixun,"MSA, Principle City",0.0,0.0,NaN,NaN
20026,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Unemployed,mlyzmhmf,"MSA, Principle City",0.0,0.0,NaN,NaN
20027,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,Non-MSA,1.0,0.0,atmlpfrs,xqwwgdyp
20028,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,kbazzjca,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea


In [8]:
y_train

0        0
1        1
2        1
3        0
4        1
        ..
20025    1
20026    0
20027    0
20028    0
20029    0
Name: seasonal_vaccine, Length: 20030, dtype: int64

Begin Pipeline Construction

In [9]:
x_train.columns

Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')

In [10]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20030 entries, 0 to 20029
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 19963 non-null  float64
 1   h1n1_knowledge               19943 non-null  float64
 2   behavioral_antiviral_meds    19974 non-null  float64
 3   behavioral_avoidance         19873 non-null  float64
 4   behavioral_face_mask         20016 non-null  float64
 5   behavioral_wash_hands        19994 non-null  float64
 6   behavioral_large_gatherings  19960 non-null  float64
 7   behavioral_outside_home      19972 non-null  float64
 8   behavioral_touch_face        19932 non-null  float64
 9   doctor_recc_h1n1             18395 non-null  float64
 10  doctor_recc_seasonal         18395 non-null  float64
 11  chronic_med_condition        19313 non-null  float64
 12  child_under_6_months         19425 non-null  float64
 13  health_worker   

In [11]:
#labels

#already encoded in survey responses ordinally, values generally range from 0 to 5
ordinal_feat = ["h1n1_concern","h1n1_knowledge",'opinion_h1n1_vacc_effective','opinion_h1n1_risk',
                'opinion_h1n1_sick_from_vacc','opinion_seas_vacc_effective','opinion_seas_risk',
                'opinion_seas_sick_from_vacc','household_adults','household_children']
#features that are ordinal in nature, but are recorded as strings
ordinal_e_feat = ["age_group", "education"]
#categorical and binary features recorded in the dataset. Nulls represent "prefer not to answer"- 
#-type responses in compliance with human subjects research ethics requirements. Thus, all binary 
#features are now categorical to account for trends in refusal behavior in the model.
cat_feat = ['behavioral_antiviral_meds', 'behavioral_avoidance',
           'behavioral_face_mask', 'behavioral_wash_hands',
           'behavioral_large_gatherings', 'behavioral_outside_home',
           'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
           'chronic_med_condition', 'child_under_6_months', 'health_worker',
           'health_insurance', 'race', 'sex', 'income_poverty', 'marital_status',
           'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry',
           'employment_occupation']

In [12]:
#subpipelines
ordinal_impute = Pipeline(steps=[('impute', SimpleImputer(strategy="most_frequent"))])
ordinal_encoding = Pipeline(steps=[('impute', SimpleImputer(strategy="most_frequent")),
                                  ("ord_encode", OrdinalEncoder())
                                  ])
#mapping binary values to yes/no to resolve errors with one_hot_encoder
def mapper(data):
    data = data.applymap(lambda x: "yes" if (x==1) else x or "no" if (x==0) else x)
    data.astype(str)
    return data
mappy = FunctionTransformer(func=mapper)    
#map_binary = Pipeline(steps=[("map_bin", mappy)])

one_hot_encoder = Pipeline(steps=[("map_bin", mappy),
                                  ('impute', SimpleImputer(strategy="constant",fill_value="no_response")),
                                  ("ohe", OneHotEncoder(sparse=False, handle_unknown="ignore"))
                                 ])


In [13]:
#columntransformer
col_tr = ColumnTransformer(transformers=[#("map", map_binary, cat_feat),
                                         ("ord_imp", ordinal_impute, ordinal_feat), 
                                         ("ord_enc", ordinal_encoding, ordinal_e_feat),
                                         ("imp_ohe", one_hot_encoder, cat_feat)],
                          sparse_threshold=0)

In [14]:
col_tr.fit_transform(x_train)

array([[1., 1., 4., ..., 0., 0., 0.],
       [2., 1., 3., ..., 0., 0., 0.],
       [0., 0., 4., ..., 0., 0., 0.],
       ...,
       [2., 1., 4., ..., 1., 0., 0.],
       [2., 1., 1., ..., 0., 0., 0.],
       [3., 1., 4., ..., 0., 0., 0.]])

### Construct Initial Model

Initial model will be a voting classifier composed of a random forest estimator and a naive bayes estimator. A grid search will then be performed to find the best hyperparameters for each base estimator as well as the overall weights of the voting classifier. 

In [15]:
#import estimators
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, plot_confusion_matrix, plot_roc_curve, classification_report

In [16]:
#instantiate base estimators
rfc = RandomForestClassifier()
cat_nb = CategoricalNB()

In [17]:
#instantiate meta classifier
vc = VotingClassifier(estimators=[("rfc", rfc),
                                 ("cat_nb", cat_nb)])

In [18]:
initial_model = Pipeline(steps=[("col_tr", col_tr),
                   ("vc", vc)])
initial_model.fit(x_train,y_train)

Pipeline(steps=[('col_tr',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('ord_imp',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'opinion_h1n1_vacc_effective',
                                                   'opinion_h1n1_risk',
                                                   'opinion_h1n1_sick_from_vacc',
                                                   'opinion_seas_vacc_effective',
                                                   'opinion_seas_risk',
                                                   'opinion_seas_sick_from_vacc',...
                                                   'doctor

In [19]:
cross_val_score(estimator=initial_model, X=x_train,y=y_train)
classification_report(y_train, initial_model.predict(x_train))

'              precision    recall  f1-score   support\n\n           0       0.80      1.00      0.89     10638\n           1       1.00      0.72      0.84      9392\n\n    accuracy                           0.87     20030\n   macro avg       0.90      0.86      0.86     20030\nweighted avg       0.89      0.87      0.87     20030\n'